<a href="https://colab.research.google.com/github/BirukovAlex/neto_Python/blob/main/%D0%94%D0%97_Dogs_VS_Cats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Все изображения перенес на Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Импортируем библиотеки

In [ ]:
import tensorflow as tf
import numpy as np
import os
from glob import glob
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from sklearn.model_selection import train_test_split
import time

Устанавливаем параметры

In [ ]:
IMG_SIZE = (128, 128)
BATCH_SIZE = 32
EPOCHS = 20
LEARNING_RATE = 1e-3

Загружаем данные

In [ ]:
train_files = glob('/content/drive/MyDrive/data/train/*.jpg')
test_files = glob('/content/drive/MyDrive/data/test/*.jpg')

print(f" Найдено тренировочных изображений: {len(train_files)}")
print(f" Найдено тестовых изображений: {len(test_files)}")

 Найдено тренировочных изображений: 25001
 Найдено тестовых изображений: 12500


Делим на train/val

In [ ]:
train_files, val_files = train_test_split(
    train_files,
    test_size=0.2,
    random_state=42,
    shuffle=True
)

print(f" Обучающая выборка: {len(train_files)} изображений")
print(f" Валидационная выборка: {len(val_files)} изображений")


 Обучающая выборка: 20000 изображений
 Валидационная выборка: 5001 изображений


In [ ]:
# Функция создания dataset
def create_dataset(files, batch_size=BATCH_SIZE, is_training=True, dataset_name="dataset"):
    print(f" Создание {dataset_name}...")
    images = []
    labels = []
    skipped_files = 0

    for i, path in enumerate(files):
        if i % 500 == 0 and i > 0:
            print(f"   Обработано {i}/{len(files)} изображений...")

        import cv2
        img = cv2.imread(path)
        if img is None:
            skipped_files += 1
            continue

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, IMG_SIZE)
        img = preprocess_input(img.astype(np.float32))

        images.append(img)

        filename = os.path.basename(path).lower()
        if 'dog' in filename:
            labels.append(1.0)
        else:
            labels.append(0.0)

    print(f" {dataset_name}: {len(images)} изображений, пропущено: {skipped_files}")

    dataset = tf.data.Dataset.from_tensor_slices((np.array(images), np.array(labels)))

    if is_training:
        dataset = dataset.shuffle(len(images))

    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)

    return dataset

In [ ]:
# Создаем модель
def create_model():
    print("\n Создание модели MobileNetV2...")
    base_model = MobileNetV2(
        weights='imagenet',
        include_top=False,
        input_shape=(*IMG_SIZE, 3)
    )
    base_model.trainable = False
    print(" Базовые слои заморожены")

    model = tf.keras.Sequential([
        base_model,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    print(" Модель скомпилирована")
    return model

In [ ]:
# Создаем datasets
print("\n Подготовка datasets...")
train_dataset = create_dataset(train_files[:2000], BATCH_SIZE, True, "TRAIN dataset")  # 2000 для скорости
val_dataset = create_dataset(val_files[:500], BATCH_SIZE, False, "VAL dataset")       # 500 для валидации

print(f" Размер train dataset: {len(list(train_dataset))} батчей по {BATCH_SIZE} изображений")
print(f" Размер val dataset: {len(list(val_dataset))} батчей по {BATCH_SIZE} изображений")


 Подготовка datasets...
 Создание TRAIN dataset...
   Обработано 500/2000 изображений...
   Обработано 1000/2000 изображений...
   Обработано 1500/2000 изображений...
 TRAIN dataset: 2000 изображений, пропущено: 0
 Создание VAL dataset...
 VAL dataset: 500 изображений, пропущено: 0
 Размер train dataset: 63 батчей по 32 изображений
 Размер val dataset: 16 батчей по 32 изображений


In [ ]:
# Callbacks
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True,
        verbose=1
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        verbose=1
    )
]

In [ ]:
# Создаем модель
model = create_model()
# Выводим архитектуру
print("\n Архитектура модели:")
model.summary()


 Создание модели MobileNetV2...
 Базовые слои заморожены
 Модель скомпилирована

 Архитектура модели:


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_128            │ (None, 4, 4, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,422,081 (9.24 MB)

 Trainable params: 164,097 (641.00 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

Предварительное обучение на малых выборках

In [ ]:
# Обучение
print(f"\n НАЧАЛО ОБУЧЕНИЯ")
print(f"   Эпох: {EPOCHS}")
print(f"   Размер батча: {BATCH_SIZE}")
print(f"   Learning rate: {LEARNING_RATE}")
print(f"   Размер изображения: {IMG_SIZE}")

start_time = time.time()

history = model.fit(
    train_dataset,
    epochs=EPOCHS,
    validation_data=val_dataset,
    callbacks=callbacks,
    verbose=1
)

training_time = time.time() - start_time
print(f" Обучение завершено за {training_time/60:.1f} минут")


 НАЧАЛО ОБУЧЕНИЯ
   Эпох: 20
   Размер батча: 32
   Learning rate: 0.001
   Размер изображения: (128, 128)
Epoch 1/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 38s 497ms/step - accuracy: 0.8856 - loss: 0.2416 - val_accuracy: 0.9740 - val_loss: 0.0889 - learning_rate: 0.0010
Epoch 2/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 29s 461ms/step - accuracy: 0.9772 - loss: 0.0701 - val_accuracy: 0.9780 - val_loss: 0.0640 - learning_rate: 0.0010
Epoch 3/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 29s 462ms/step - accuracy: 0.9869 - loss: 0.0382 - val_accuracy: 0.9800 - val_loss: 0.0602 - learning_rate: 0.0010
Epoch 4/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 29s 469ms/step - accuracy: 0.9884 - loss: 0.0317 - val_accuracy: 0.9760 - val_loss: 0.0680 - learning_rate: 0.0010
Epoch 5/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 47s 569ms/step - accuracy: 0.9905 - loss: 0.0252 - val_accuracy: 0.9760 - val_loss: 0.0610 - learning_rate: 0.0010
Epoch 6/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 363ms/step - accuracy: 0.9903 - loss: 0.0236
Epoch 6: ReduceLROnPlateau reducing learn

Оценка на малых выборках

In [ ]:
# Оценка
print("\n Оценка качества модели...")
val_loss, val_accuracy = model.evaluate(val_dataset, verbose=0)

print(f"\n ФИНАЛЬНЫЕ РЕЗУЛЬТАТЫ:")
print(f"   Валидационная точность: {val_accuracy:.4f}")
print(f"   Валидационные потери (Log Loss): {val_loss:.4f}")

if val_loss < 0.3:
    print(" Log Loss < 0.3 ДОСТИГНУТ!")
elif val_loss < 0.5:
    print(" Хороший результат! Log Loss < 0.5")
elif val_loss < 1.0:
    print("Средний результат, нужно больше эпох")
else:
    print("Нужна донастройка гиперпараметров")


 Оценка качества модели...

 ФИНАЛЬНЫЕ РЕЗУЛЬТАТЫ:
   Валидационная точность: 0.9800
   Валидационные потери (Log Loss): 0.0602
 Log Loss < 0.3 ДОСТИГНУТ!


In [ ]:
# Сохраняем модель
print("\nСохранение модели...")
model.save('dogs_cats_final.h5')
print(" Модель сохранена как 'dogs_cats_final.h5'")


Сохранение модели...
 Модель сохранена как 'dogs_cats_final.h5'


In [ ]:
# Предсказания на тестовых данных
print("\n Генерация предсказаний для тестовых данных...")
test_dataset = create_dataset(test_files, BATCH_SIZE, False, "TEST dataset")
predictions = model.predict(test_dataset, verbose=1)
print(f" Сгенерировано {len(predictions)} предсказаний")


 Генерация предсказаний для тестовых данных...
 Создание TEST dataset...
   Обработано 500/12500 изображений...
   Обработано 1000/12500 изображений...
   Обработано 1500/12500 изображений...
   Обработано 2000/12500 изображений...
   Обработано 2500/12500 изображений...
   Обработано 3000/12500 изображений...
   Обработано 3500/12500 изображений...
   Обработано 4000/12500 изображений...
   Обработано 4500/12500 изображений...
   Обработано 5000/12500 изображений...
   Обработано 5500/12500 изображений...
   Обработано 6000/12500 изображений...
   Обработано 6500/12500 изображений...
   Обработано 7000/12500 изображений...
   Обработано 7500/12500 изображений...
   Обработано 8000/12500 изображений...
   Обработано 8500/12500 изображений...
   Обработано 9000/12500 изображений...
   Обработано 9500/12500 изображений...
   Обработано 10000/12500 изображений...
   Обработано 10500/12500 изображений...
   Обработано 11000/12500 изображений...
   Обработано 11500/12500 изображений...
   

Submission for Kaggle

In [ ]:
# Сохраняем предсказания для Kaggle
print("\n Создание файла для Kaggle...")
import re
with open('submission.csv', 'w') as f:
    f.write('id,label\n')
    for i, path in enumerate(test_files):
        file_id = re.search(r'(\d+)\.jpg$', path).group(1)
        f.write(f'{file_id},{predictions[i][0]:.6f}\n')

print(" Файл 'submission.csv' создан")
print("\n ВСЕ ЗАДАЧИ ВЫПОЛНЕНЫ!")
print(" Результаты обучения:")
print(f"   - Лучший Val Loss: {val_loss:.4f}")
print(f"   - Лучшая Val Accuracy: {val_accuracy:.4f}")
print(" Файлы для отправки:")
print("   - dogs_cats_final.h5 (модель)")
print("   - submission.csv (предсказания для Kaggle)")


 Создание файла для Kaggle...
 Файл 'submission.csv' создан

 ВСЕ ЗАДАЧИ ВЫПОЛНЕНЫ!
 Результаты обучения:
   - Лучший Val Loss: 0.0602
   - Лучшая Val Accuracy: 0.9800
 Файлы для отправки:
   - dogs_cats_final.h5 (модель)
   - submission.csv (предсказания для Kaggle)


Ниже написан код для обучения на полных данных. обучение на полных данных не проводилось - недостаточно ОЗУ (процесс прерывается на подготовке датасетов).

In [ ]:
print("\n Запуск обучения на ВСЕХ данных...")
print(" Подготовка полных datasets...")

full_train_dataset = create_dataset(train_files, BATCH_SIZE, True, "FULL TRAIN dataset")
full_val_dataset = create_dataset(val_files, BATCH_SIZE, False, "FULL VAL dataset")

print(f" Полное обучение: {len(train_files)} train, {len(val_files)} val")

In [ ]:
import time
start_time = time.time()

history_full = model.fit(
        full_train_dataset,
        epochs=10,
        validation_data=full_val_dataset,
        callbacks=callbacks,
        verbose=1
)

full_training_time = time.time() - start_time
print(f" Полное обучение завершено за {full_training_time/60:.1f} минут")

final_val_loss, final_val_accuracy = model.evaluate(full_val_dataset, verbose=0)
print(f"\n ФИНАЛЬНЫЕ РЕЗУЛЬТАТЫ НА ВСЕХ ДАННЫХ:")
print(f"   Val Loss: {final_val_loss:.4f}")
print(f"   Val Accuracy: {final_val_accuracy:.4f}")